In [5]:
import pickle5 as pickle
import random
import numpy as np

with open('../tokens', 'rb') as pickle_file:
    tokenized_sent = pickle.load(pickle_file)

random.shuffle(tokenized_sent)

def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

test_tokens = tokenized_sent[:10000]
train_tokens = [(' '.join(token[2]), token[1]) for token in tokenized_sent[10000:]]

In [6]:
!pip install sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Defaulting to user installation because normal site-packages is not writeable


In [7]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [8]:
sentence_embeddings = model.encode([i[0] for i in train_tokens])

In [9]:
query = ''.join(test_tokens[0][2])
query_vec = model.encode([query])[0]

In [10]:
for sent in train_tokens[:10]:
  sim = cosine(query_vec, model.encode([sent])[0])
  print("Sentence = ", sent, "; similarity = ", sim)

Sentence =  ('uwb technology for wireless communications has been received attention for many years uwb antennas require characteristics that maintain good impedance matching and radiation patterns over a wide frequency range the uwb frequency range includes the already existing narrow band which can cause interference to the uwb system to minimize this interference we propose a notch band antenna that removes the 5g ghz ghz band using a split slit', '5g') ; similarity =  0.07734063
Sentence =  ('generalized sparse nonnegative matrix factorization snmf has been proven useful in extracting information and representing sparse data with various types of probabilistic distributions from industrial applications recommender systems and social current solution approaches for generalized snmf are based on the manipulation of whole sparse matrices and factor matrices which will result in intermediate these approaches can not describe the and sparse matrices in mainstream industrial and big data

In [11]:
import numpy as np
import pandas as pd
results = []
for token in tokenized_sent:
    class_token = token[1]
    token_embedding = model.encode([' '.join(token[2])])[0] 

    row = np.append(token_embedding, [class_token])
    results.append(row)

df = pd.DataFrame(results)
df.to_csv('../dataframes/sentence_bert_embedding.csv', index=False, encoding='utf-8')